### IS - Brand position
Finding Product position on Social Network (PPSN)

In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import itertools
import math
import string
import json
import pymongo
from itertools import groupby

In [2]:
import gensim
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=pd.core.common.SettingWithCopyWarning)

C:\ProgramData\Anaconda3\lib\site-packages\past\types\oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable
C:\ProgramData\Anaconda3\lib\site-packages\past\builtins\misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping


In [3]:
import time
from datetime import datetime
today = datetime.now()

In [4]:
#!pip install pythainlp
import pythainlp
from pythainlp import Tokenizer
from pythainlp.util import normalize
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus.common import thai_words
from pythainlp import sent_tokenize
from pythainlp.corpus import thai_stopwords
from pythainlp.util import dict_trie
from pythainlp.tag import pos_tag

In [5]:
# https://thainlp.org/pythainlp/docs/2.0/api/tag.html
from pythainlp.spell import correct
from pythainlp.tag import pos_tag

#https://thainlp.org/pythainlp/docs/2.0/api/transliterate.html
from pythainlp.transliterate import romanize

In [6]:
import statistics as stat
import matplotlib.pyplot as plt
import seaborn as sns

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [7]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [8]:
from IPython.display import clear_output

In [9]:
# Files definition
root_path = 'Datasource'

comment_file = []
comment_file.append(root_path+"/comment_meiji.csv")
comment_file.append(root_path+"/comment_DDCF.csv")

#comment_file = []
#comment_file.append("comment_meiji.csv")
#comment_file.append("comment_DDCF.csv")

comment_nlp_file = root_path+"/comment_nlpToken.csv"
comment_cooc_freq_file = root_path+"/comment_cooc_freq.xlsx"
comment_cooc_lift_file = root_path+"/comment_cooc_Lift.xlsx"

#comment_nlp_file = "comment_nlpToken.csv"
#comment_cooc_nlp_file = "comment_cooc.xlsx"

In [10]:
# Connect MongoDB
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["NIDA_PPSN_PRD"]
col_thread = mydb["NIDA_PPSN_THREAD"]
col_comment = mydb["NIDA_PPSN_COMMENT"]

### 1.Clean data & NLP Tokenized

In [11]:
def checkStopwords(w):
    returnList = []
    stopwords = set(thai_stopwords())
    stopwords.remove('ไม่')
    stopwords.remove('ได้')
    stopwords.remove('สูง')
    for i in w:  # comment
        if i not in stopwords:
            returnList.append(i)
    return returnList

In [12]:
def normThai(w):
    returnList = []
    for i in w:
        returnList.append(normalize(i).lower())
    return returnList

In [13]:
list_punc = []
for c in string.punctuation:
    list_punc.append(c)

In [14]:
df_textList1 = pd.read_csv(comment_file[0],header=None)
df_textList2 = pd.read_csv(comment_file[1],header=None)

In [15]:
df_textList1.insert(0,'Table','Meiji')
df_textList1.columns = ['Table','URLs', 'text', 'Retrived_date']
df_textList2.insert(0,'Table','DDFC')
df_textList2.columns = ['Table','URLs', 'text', 'Retrived_date']

In [16]:
pd.DataFrame(df_textList1.URLs.value_counts()).describe().T

,count,mean,std,min,25%,50%,75%,max
URLs,1000.0,43.049,53.682105,3.0,11.0,21.0,49.0,351.0


In [17]:
print('meiji comment:',df_textList1.shape[0])
print('meiji URLs ก่อนตัด:',len(df_textList1.URLs.unique()))

meiji comment: 43049
meiji URLs ก่อนตัด: 1000


In [18]:
pd.DataFrame(df_textList2.URLs.value_counts()).describe().T

,count,mean,std,min,25%,50%,75%,max
URLs,517.0,44.756286,57.056773,1.0,10.0,19.0,53.0,328.0


In [19]:
print('DDCF comment:',df_textList2.shape[0])
print('DDCF URLs ก่อนตัด:',len(df_textList2.URLs.unique()))

DDCF comment: 23139
DDCF URLs ก่อนตัด: 517


In [20]:
df_Corpus = pd.concat([df_textList1,df_textList2],ignore_index=True)

In [21]:
print('All comment:',df_Corpus.shape[0])
print('All URLs ก่อนตัด:',len(df_Corpus.URLs.unique()))

All comment: 66188
All URLs ก่อนตัด: 1517


In [22]:
missing = df_Corpus.isnull().sum()
print(missing)

Table               0
URLs                0
text             4703
Retrived_date       0
dtype: int64


In [23]:
df_Corpus.tail()

,Table,URLs,text,Retrived_date
66183,DDFC,https://pantip.com/topic/30651064,มีหลายยี่ห้อนะค่ะ พวกจิตรดา ไทยเดนมาร์ค โฟร์โม...,2020-10-27 13:54:33.920930
66184,DDFC,https://pantip.com/topic/30651064,เดี๋ยวนมรสจืด ส่วนใหญ่ก็นมโคแท้ 100%ทั้งนั้นแห...,2020-10-27 13:54:33.920930
66185,DDFC,https://pantip.com/topic/30651064,ตามกฏหมายเอง นมโคสดแท้ 100% กับนมโคแท้ 100% ไม...,2020-10-27 13:54:33.920930
66186,DDFC,https://pantip.com/topic/30651064,NaN,2020-10-27 13:54:33.920930
66187,DDFC,https://pantip.com/topic/30651064,NaN,2020-10-27 13:54:33.920930


In [24]:
seqNum = list(range(1,df_Corpus.shape[0]+1))
df_Corpus.insert(0,'commentId',seqNum)
df_Corpus.columns = ['commentId','Table','URLs', 'text', 'Retrived_date']

In [25]:
df_Corpus.tail()

,commentId,Table,URLs,text,Retrived_date
66183,66184,DDFC,https://pantip.com/topic/30651064,มีหลายยี่ห้อนะค่ะ พวกจิตรดา ไทยเดนมาร์ค โฟร์โม...,2020-10-27 13:54:33.920930
66184,66185,DDFC,https://pantip.com/topic/30651064,เดี๋ยวนมรสจืด ส่วนใหญ่ก็นมโคแท้ 100%ทั้งนั้นแห...,2020-10-27 13:54:33.920930
66185,66186,DDFC,https://pantip.com/topic/30651064,ตามกฏหมายเอง นมโคสดแท้ 100% กับนมโคแท้ 100% ไม...,2020-10-27 13:54:33.920930
66186,66187,DDFC,https://pantip.com/topic/30651064,NaN,2020-10-27 13:54:33.920930
66187,66188,DDFC,https://pantip.com/topic/30651064,NaN,2020-10-27 13:54:33.920930


In [26]:
# Drop NA & banned comments
df_Corpus = df_Corpus.dropna()
df_Corpus = df_Corpus[df_Corpus["text"].str.find('ความคิดเห็นนี้ถูกลบ')!= 0]
df_Corpus = df_Corpus[df_Corpus["text"].str.find('แก้ไขข้อความเมื่อ')!= 0]
df_Corpus = df_Corpus.set_index('commentId')

In [27]:
missing = df_Corpus.isnull().sum()
print(missing)

Table            0
URLs             0
text             0
Retrived_date    0
dtype: int64


In [28]:
print('จำนวน comment หลังตัด:',df_Corpus.shape[0])
print('จำนวนกระทู้หลังตัด:',len(df_Corpus.URLs.unique()))

จำนวน comment หลังตัด: 60545
จำนวนกระทู้หลังตัด: 1514


In [29]:
df_Corpus.insert(4,'token_text',None)
#df_Corpus.insert(5,'token_sound',None)
#df_Corpus.insert(6,'token_tag',None)
df_Corpus.tail()

,Table,URLs,text,Retrived_date,token_text
commentId,,,,,
66181,DDFC,https://pantip.com/topic/30651064,นมผง คือเอานมดิบดีๆ มาคัดไขมันนมและอื่นๆออกไปใ...,2020-10-27 13:54:33.920930,None
66182,DDFC,https://pantip.com/topic/30651064,นมสด รสชาติมันดีนะ\nเพราะมีปริมาณไขมันสูง ดังน...,2020-10-27 13:54:33.920930,None
66184,DDFC,https://pantip.com/topic/30651064,มีหลายยี่ห้อนะค่ะ พวกจิตรดา ไทยเดนมาร์ค โฟร์โม...,2020-10-27 13:54:33.920930,None
66185,DDFC,https://pantip.com/topic/30651064,เดี๋ยวนมรสจืด ส่วนใหญ่ก็นมโคแท้ 100%ทั้งนั้นแห...,2020-10-27 13:54:33.920930,None
66186,DDFC,https://pantip.com/topic/30651064,ตามกฏหมายเอง นมโคสดแท้ 100% กับนมโคแท้ 100% ไม...,2020-10-27 13:54:33.920930,None


เก็บเฉพาะหัวกระทู้

In [30]:
df_thread = df_Corpus.copy()
df_thread = df_thread[['URLs','text']]
df_thread.drop_duplicates(subset ="URLs", keep = 'first', inplace = True)
df_thread.insert(2,'token_text',None)
df_thread.insert(3,'mention_product',None)
df_thread.insert(4,'mention_brand',None)
df_thread.tail()

,URLs,text,token_text,mention_product,mention_brand
commentId,,,,,
65991,https://pantip.com/topic/30007179,กำลังจะเป็นคุณแม่มือใหม่ค่ะ ทำตัวไม่ถูกเลยค่ะ\...,None,None,None
66015,https://pantip.com/topic/30020409,พรุ่งนี้ลูกชายอายุครบ 1 ขวบ 3 เดือน หม่ำนมแม่จ...,None,None,None
66044,https://pantip.com/topic/30049123,"เพื่อนๆทุกคนคงคุ้นเคยกับคำถามนี้แล้วทุกคน... ""...",None,None,None
66074,https://pantip.com/topic/30011023,แตกกระทู้มาจาก www.pantip.com/topic/30007146/\...,None,None,None
66132,https://pantip.com/topic/30651064,อยากรู้ว่าปัจจุบันพวกนมกล่องทึ่ขายใน 7-11 หรือ...,None,None,None


## 2.ตัดคำและเก็บลง MongoDB (ใช้ insight จาก jupyter งานก่อน)

https://www.thainlp.org/pythainlp/tutorials/notebooks/pythainlp-get-started.html#Thai-Characters

In [31]:
garbage_char = ['',' ','🥺','\r','\n','\r\n','\xa0','www','application','ทางกฎหมาย','html'
                ,'http','https','url','com','pantip','topic','_','–','พันทิป','^^'
                ,'นะค่ะ','นะคะ','น่ะค่ะ','ฯ','ๆ','แฮ่','เย่ๆๆ','ว๊าว','ฮิ้วววววววววว','Number','กก'
                ,'เมนู','Menu','Net','net','สาขา','บาท','ราคา','(',')','ฯ','ๆ','กก','อันนี้','%s','😆', '🤣','😢'
                ,'😏','😂','😿','ววว','xx','อิอิ','แย้ววว','นะคะ','นะค่ะ','://','มกราคม', 'กุมภาพันธ์'
                ,'มีนาคม', 'เมษายน','พฤษภาคม', 'มิถุนายน', 'กรกฎาคม', 'สิงหาคม','กันยายน', 'ตุลาคม'
                ,'พฤศจิกายน', 'ธันวาคม']
garbage_char  = garbage_char + list_punc

In [32]:
print('จำนวน comment หลังตัด:',df_Corpus.shape[0])
print('จำนวนกระทู้หลังตัด:',len(df_Corpus.URLs.unique()))

จำนวน comment หลังตัด: 60545
จำนวนกระทู้หลังตัด: 1514


In [33]:
custom_list = ['ดัชมิลล์','ดัชมิล','dutch mill','dutchmilk','ดัชชี่','เมจิ','ซีพีเมจิ','ซีพี-เมจิ','cp-meiji','betagen','บีทาเก้น','บีทาเกน'
               ,'โฟร์โมสต์','โฟร์โมส','โฟโมสต์','โฟโมต','โฟรโมสต์','โชคชัย','แดรี่โฮม','dairy home','เดรี่โฮม','อืมม มิลค์'
               ,'Umm! Milk','เอ็มมิลค์','mmilk','เอ็มมิ้ลค์','แมคโนเลีย','นมฟาร์มโชคชัย','นมไทย-เดนมาร์ค','ไทยเดนมาร์ค','วัวแดง','หนองโพ','คาเนชั่น'
               ,'สตอเบอร์รี่','สตรอว์เบอร์รี','รสสตรอเบอรี่','สตรอเบอรี่','สตรอวเบอรี่','สตอเบอรี่','สตรอเบอรี่','รสกาแฟ'
               ,'ชอคโกแล็ต','ช็อกโกแลต','ช็อคโกแลต','ช๊อกโกแลต','ช็อคโกแล็ต','ช็อกโกเลต','มิ้นท์ชอค','มิ้นต์ช้อก','รสกล้วย'
               ,'รสกาแฟ','รสหวาน','รสจืด','ไขมันต่ำ','ไขมัน 0%','low fat','0% fat','ขาดมันเนย','พร่องมันเนย','ไฮแคลเซียม'
               ,'high protein','Hi Protein','HIgh Protein','Whey Formula','วานิลลา'
               ,'ไฮโปรตีน','เวย์โปรตีน','whey','เวย์','อัลมอนด์','ซีเล็ค','ซีเล็คท์','กล้วยหอม','ไวท์คอฟฟี่','คาปูชิโน่','อเมริกาโน่'
               ,'grass fed','free lactose','lactose free','พาสเจอไรซ์','พาสเจอร์ไรซ์','พาสเจอร์ไรส์','พาซเจอไรซ์','พาสเจอร์ไลท์'
               ,'เมจิโกลด์ แม็กซ์','เมจิโกลด์','เมจิโกล์ดแม็กซ์','เมจิ บัลแกเรีย','เมจิบัลแกเรีย','นมเปรี้ยว','รสธรรมชาติ','รสกลมกล่อม'
               ,'โยเกิต','โยเกิรต','โยเกิร์ตพร้อมดื่ม','น้ำตาลมะพร้าว','นมฮอกไกโด','เบดไทม์','ฟรีแลคโตส','แลคโตสฟรี','ริชเอสเพสโซ่'
               ,'bedtime','bed time','เบดไทม์','ดาร์คช็อกโกแลต','ดาร์กช็อก','ดาร์กช็อกโกแลต'
               ,'คาราเมล','ช็อกโกมอลต์','เมล่อน','ชาเขียวมัจฉะ','ฝาน้ำเงิน','ผลิตภันท์นม'
               ,'7-Eleven','เซเว่น อีเลฟเว่น','เซเว่นอีเลฟเว่น','เซเว่น','เซเวน','7/11','7-11'
               ,'ท็อปส์','ทอปส์','ท้อปส์','ท๊อปส์','แมคโคร','แม็คโคร','โลตัส','บิ๊กซี','bigc','Golden place','Big C'
               ,'ขายไม่ดี','แพคคู่','ค่าจัดส่ง','shelf life','พนักงานขายนม'               
               ,'ซื้อประจำ','ซื้อไม่ได้','ของแถม','ราคาสูง','น้ำนมโค'
               ,'แพ้นม','แพ้นมวัว','เล่นเวท','นำ้หนัก','คุณแม่มือใหม่','นมอุ่น','ชานม','กินนม','ดื่มนม','ท้องเสีย','ขี้แตก'
               ,'คีเฟอร์นม','พันทิป','ร้านนม','ซื้อไม่ได้','ลูกสุนัข','ลูกหมา','คายทิ้ง','เจมส์ จิ','เจมส์จิ','สตอรี่'
              ]
words = set(thai_words()).union(set(custom_list))
#custom_tokenizer = Tokenizer(engine='ulmfit', custom_dict=words, keep_whitespace=False)
trie = dict_trie(dict_source=words)

In [34]:
#https://github.com/PyThaiNLP/pythainlp-corpus/releases/tag/wiki_lm_lstm-v0.32

#from pythainlp.ulmfit import (replace_rep_after,fix_html,ungroup_emoji,replace_wrep_post,remove_space)

In [35]:
def tokenize_custom(x, cusDict):
    wtkn = word_tokenize(x, engine="newmm", custom_dict=cusDict, keep_whitespace=False)
    wtkn = checkStopwords(wtkn)
    wtkn = [x for x in wtkn if len(x) >= 2]
    wtkn = [y for y in wtkn if y not in garbage_char]
    wtkn = [s for s in wtkn if not s.isdigit()]
    wtkn = normThai(wtkn)
    return wtkn

In [36]:
def create_InsertMany_Thread(df):
    listofdict = []
    for c,lx in enumerate(df.URLs,0):
        info = {
            "URLs": lx,
            "text": df.text[c],
            "token_text":df.token_text[c],
            "t_mention_product": df.t_mention_product[c],
            "t_mention_brand": df.t_mention_brand[c]
        }
        listofdict.append(info)
    return listofdict

In [37]:
def create_InsertMany_comments(df):
    listofdict = []
    for c,lx in enumerate(df.URLs,0):
        info = {
            "commentId":int(df.commentId[c]),
            "Table": df.Table[c],
            "URLs": lx,
            "text": df.text[c],
            "Retrived_date": df.Retrived_date[c],
            "token_text":df.token_text[c],
            "cmt_mention_product": df.cmt_mention_product[c],
            "cmt_mention_brand": df.cmt_mention_brand[c]
        }
        listofdict.append(info)
    return listofdict

In [38]:
def generalize_keyword(wtkn):  
    ret_wtkn = wtkn
    for i,each in enumerate(ret_wtkn,0):
        if each in ['ดัชมิลล์','ดัชมิล','dutch mill','dutchmill','dutchmilk','ดัชชี่']:
            ret_wtkn[i] = 'ดัชมิลล์'
        elif each  in ['เมจิ','ซีพีเมจิ','ซีพี-เมจิ','cp-meiji','meiji','meji','miji']:
            ret_wtkn[i] = 'เมจิ'
        elif each  in ['โฟร์โมสต์','โฟร์โมส','โฟโมสต์','โฟรโมสต์','โฟโมต','foremost']:
            ret_wtkn[i] = 'โฟร์โมสต์'
        elif each  in ['แดรี่โฮม','dairy home','เดรี่โฮม']:
            ret_wtkn[i] = 'แดรี่โฮม'
        elif each  in ['โชคชัย','อืมม มิลค์','umm milk','นมฟาร์มโชคชัย']:
            ret_wtkn[i] = 'โชคชัย'
        elif each  in ['เอ็มมิลค์','mmilk','เอ็มมิ้ลค์']:
            ret_wtkn[i] = 'เอ็มมิลค์'
        elif each  in ['นมไทย-เดนมาร์ค','ไทยเดนมาร์ค','วัวแดง']:
            ret_wtkn[i] = 'ไทยเดนมาร์ค'
        elif each  in ['สตอเบอร์รี่','สตรอว์เบอร์รี','รสสตรอเบอรี่','สตอเบอรี่','สตรอเบอรี่','สตรอวเบอรี่']:
            ret_wtkn[i] = 'สตรอว์เบอร์รี'
        elif each  in ['ชอคโกแล็ต','ช็อกโกแลต','ช็อคโกแลต','ช๊อกโกแลต','ช็อคโกแล็ต','ช็อกโกเลต','ช็อค']:
            ret_wtkn[i] = 'ช็อกโกแลต'
        elif each  in ['มิ้นท์ชอค','มิ้นต์ช้อก','ช็อคโกแลตมินต์','มินต์ช็อคโกแลต']:
            ret_wtkn[i] = 'ช็อกโกแลตมินต์'
        elif each  in ['ไขมันต่ำ','low fat','พร่องมันเนย']:
            ret_wtkn[i] = 'ไขมันต่ำ'
        elif each  in ['ไขมัน 0%','0% fat','ขาดมันเนย']:
            ret_wtkn[i] = 'ไขมัน 0%'
        elif each  in ['high protein','hi protein','ไฮโปรตีน','เวย์โปรตีน']:
            ret_wtkn[i] = 'ไฮโปรตีน'
        elif each  in ['free lactose','lactose free','ฟรีแลคโตส','แลคโตสฟรี']:
            ret_wtkn[i] = 'นมฟรีแลคโตส'
        elif each  in ['พาสเจอไรซ์','พาสเจอร์ไรซ์','พาสเจอร์ไรส์','พาซเจอไรซ์','พาสเจอร์ไลท์']:
            ret_wtkn[i] = 'พาสเจอร์ไรส์'
        elif each  in ['เมจิโกลด์ แม็กซ์','เมจิโกลด์','เมจิโกล์ดแม็กซ์','gold','gold max']:
            ret_wtkn[i] = 'เมจิโกลด์'
        elif each  in ['ฝาน้ำเงิน','รสจืด']:
            ret_wtkn[i] = 'รสจืด'
        elif each  in ['ท็อปส์','ทอปส์','ท้อปส์','ท๊อปส์','tops']:
            ret_wtkn[i] = 'tops'
        elif each  in ['แมคโคร','แม็คโคร','makro']:
            ret_wtkn[i] = 'makro'
        elif each  in ['โลตัส','lotus']:
            ret_wtkn[i] = 'lotus'
        elif each  in ['บิ๊กซี','bigc','Big C']:
            ret_wtkn[i] = 'bigc'
        elif each  in ['ท้องเสีย','ขี้แตก']:
            ret_wtkn[i] = 'ท้องเสีย'
        elif each  in ['ลูกสุนัข','ลูกหมา']:
            ret_wtkn[i] = 'ลูกสุนัข'
        elif each in ['ดาร์คช็อกโกแลต','ดาร์กช็อก','ดาร์กช็อกโกแลต']:
            ret_wtkn[i] = 'ดาร์คช็อกโกแลต'
        elif each in ['bedtime','bed time','เบดไทม์']:
            ret_wtkn[i] = 'เบดไทม์'
        elif each in ['เมจิ บัลแกเรีย','เมจิบัลแกเรีย']:
            ret_wtkn[i] = 'เมจิบัลแกเรีย'
        elif each in ['โยเกิร์ต','โยเกิต','โยเกิรต','โยเกิร์ตพร้อมดื่ม']:
            ret_wtkn[i] = 'โยเกิร์ต'
        elif each in ['7-Eleven','เซเว่น อีเลฟเว่น','เซเว่นอีเลฟเว่น','เซเว่น','เซเวน','7/11','7-11']:
            ret_wtkn[i] = '7-Eleven'
        elif each in ['บีทาเก้น','บีทาเกน']:
            ret_wtkn[i] = 'บีทาเก้น'
    return ret_wtkn

In [39]:
def find_brands_product(x,catType):
    ret_wtkn = []
    brands = ['ดัชมิลล์','เมจิ','โฟร์โมสต์','โชคชัย','แดรี่โฮม','เอ็มมิลค์','แมคโนเลีย','ไทยเดนมาร์ค','หนองโพ','คาเนชั่น','บีทาเก้น','จิตรลดา']
    products = ['นม','"นม"','นมข้น','นมจืด','นมสด','กินนม','ดื่มนม','/\xa0\xa0นม','ขวดนม','นมวัว','นมกล่อง','ผลิตภันท์นม','น้ำนมโค'
                ,'สตรอว์เบอร์รี','ช็อกโกแลต','รสกาแฟ','รสหวาน','รสจืด','ไขมันต่ำ','ไขมัน 0%','ไฮโปรตีน','อัลมอนด์'
                ,'รสกล้วย','grass fed','นมฟรีแลคโตส','พาสเจอร์ไรส์','เมจิโกลด์','นมฮอกไกโด','เบดไทม์','ดาร์คช็อกโกแลต','ไฮแคลเซียม'
                ,'คาราเมล','มอลต์','เมล่อน','ชาเขียวมัจฉะ','บัลแกเรีย','โยเกิร์ต','รสธรรมชาติ','รสกลมกล่อม','โยเกิร์ต','ซากุระ']
    
    choiceList = []
    if catType == 'b':
        choiceList = brands
    else:
        choiceList = products
        
    for i,each in enumerate(x,0):
        if each in choiceList:
            ret_wtkn.append(each)
    return ret_wtkn

#### Copy ชุดคำจาก File มา Process ก่อนลง MongoDB

In [40]:
df_process_comment = df_Corpus.reset_index().copy()
df_process_thread = df_thread.reset_index().copy()

2.1) Tokenize comment

In [41]:
df_process_comment['token_text'] = df_process_comment['text'].apply(lambda x: tokenize_custom(x,trie))

In [42]:
df_process_comment.head()

,commentId,Table,URLs,text,Retrived_date,token_text
0,1,Meiji,https://pantip.com/topic/39868603,เมื่อก่อนหลายปีมาแล้ว ในเซเว่นยังมีนมเมจิรสหวา...,2020-10-26 12:40:26.135083,"[ปี, เซเว่น, ยังมี, นม, เมจิ, รสหวาน, ขาย, อยู..."
1,2,Meiji,https://pantip.com/topic/39868603,เบาหวานครับ แฮ่ๆ,2020-10-26 12:40:26.135083,[เบาหวาน]
2,3,Meiji,https://pantip.com/topic/39868603,หาซื้อไม่ได้นี่คิดได้ 2 อย่างนะ. คนไม่ค่อย...,2020-10-26 12:40:26.135083,"[หา, ซื้อไม่ได้, คิดได้, , . คน, กินกัน, ไม..."
3,4,Meiji,https://pantip.com/topic/39868603,เป็นรสที่ขายไม่ดีครับ\r\nรสพื้นฐานคือ\r\nจืด แ...,2020-10-26 12:40:26.135083,"[รส, ขายไม่ดี, รส, พื้นฐาน, จืด, แบ่ง, %,, low..."
4,5,Meiji,https://pantip.com/topic/39868603,ถ้าขายไม่ดีเค้าก็เอาออกครับ shelf life มันน้อย...,2020-10-26 12:40:26.135083,"[ขายไม่ดี, เค้า, shelf life, เวลา, หมดอายุ, ค่..."


2.2) Generalized keyword (Brand & product)

In [43]:
df_process_comment['token_text'] = df_process_comment['token_text'].apply(lambda x: generalize_keyword(x))

In [44]:
df_process_comment['cmt_mention_brand'] = df_process_comment['token_text'].apply(lambda x: find_brands_product(x,'b'))
df_process_comment['cmt_mention_product'] = df_process_comment['token_text'].apply(lambda x: find_brands_product(x,'p'))

2.3) Insert comments to DB

In [46]:
col_comment.delete_many({})
process_data = create_InsertMany_comments(df_process_comment)
col_comment.insert_many(process_data)

2.4) Tokenize head of thread

In [47]:
df_process_thread['token_text'] = df_process_thread['text'].apply(lambda x: tokenize_custom(x,trie))

2.5) Generalized keyword (Brand & product)

In [48]:
df_process_thread['token_text'] = df_process_thread['token_text'].apply(lambda x: generalize_keyword(x))

2.6) Mark domain/entity mention

In [49]:
df_process_thread['t_mention_brand'] = df_process_thread['token_text'].apply(lambda x: find_brands_product(x,'b'))
df_process_thread['t_mention_product'] = df_process_thread['token_text'].apply(lambda x: find_brands_product(x,'p'))

In [50]:
df_process_thread.head()

,commentId,URLs,text,token_text,mention_product,mention_brand,t_mention_brand,t_mention_product
0,1,https://pantip.com/topic/39868603,เมื่อก่อนหลายปีมาแล้ว ในเซเว่นยังมีนมเมจิรสหวา...,"[ปี, 7-Eleven, ยังมี, นม, เมจิ, รสหวาน, ขาย, อ...",None,None,[เมจิ],"[นม, รสหวาน, นม, รสหวาน]"
1,10,https://pantip.com/topic/39597709,ตามหัวข้อเลยค่ะ อยากรู้ว่านมเมจิที่ต้มก่อนนำไป...,"[หัวข้อ, อยากรู้, นม, เมจิ, ต้ม, ส่วนผสม, เครื...",None,None,[เมจิ],"[นม, นมสด]"
2,18,https://pantip.com/topic/39747201,วันก่อนซื้อ นมเมจิขวด 2 ลิตร มากิน สังเกตฝามัน...,"[วันก่อน, ซื้อ, นม, เมจิ, ขวด, ลิตร, กิน, สังเ...",None,None,[เมจิ],"[นม, นม]"
3,21,https://pantip.com/topic/39732615,อยากรู้ว่าMakroมีนมยี่ห้อ meiji(เมจิ) ขายมั้ยครับ,"[อยากรู้, makro, นม, ยี่ห้อ, เมจิ, เมจิ, ขาย]",None,None,"[เมจิ, เมจิ]",[นม]
4,28,https://pantip.com/topic/39005469,อยากถามว่า\r\n\r\n1.คิดค่าจัดส่งยังไงครับ\r\n2...,"[ถาม, 1., ค่าจัดส่ง, 2., ซื้อ, ได้, ส่วนลด, 3....",None,None,[],[]


2.7) Insert thread to DB

In [51]:
col_thread.delete_many({})
process_data = create_InsertMany_Thread(df_process_thread)
col_thread.insert_many(process_data)

In [52]:
del df_textList1,df_textList2,df_thread,df_Corpus,df_process_comment,df_process_thread

### 3.Process หัวกระทู้เพื่อดึง Domain (Brand & Product) ไม่ใช้โมเดลเทรนแล้วทำนาย ต้องการความถูกต้อง 100%

เคสหลุดเงื่อนไข แต่ต้องใช้
1. https://pantip.com/topic/34913724
2. https://pantip.com/topic/31113277
3. https://pantip.com/topic/30891578
4. https://pantip.com/topic/30833944
5. https://pantip.com/topic/30105850
6. https://pantip.com/topic/35794830
7. https://pantip.com/topic/35439062
8. https://pantip.com/topic/31351300

In [78]:
execption_urls = ['https://pantip.com/topic/34913724','https://pantip.com/topic/31113277','https://pantip.com/topic/30891578'
                  ,'https://pantip.com/topic/30833944','https://pantip.com/topic/30105850','https://pantip.com/topic/35794830'
                  ,'https://pantip.com/topic/35439062','https://pantip.com/topic/31351300']

In [83]:
cursor_thread = col_thread.find()
df_thr_process = pd.DataFrame(cursor_thread)

In [84]:
cursor_comment = col_comment.find()
df_cmt_process = pd.DataFrame(cursor_comment)

In [85]:
df_urls_milk = df_thr_process[(df_thr_process.t_mention_product.str.len() != 0) | (df_thr_process.URLs.isin(execption_urls))]
df_urls_milk['t_mention_brand'] = df_urls_milk['t_mention_brand'].apply(lambda x: repr(set(x)))
df_urls_milk['t_mention_product'] = df_urls_milk['t_mention_product'].apply(lambda x: repr(set(x)))

In [93]:
df_urls_milk.tail()

,_id,URLs,text,token_text,t_mention_product,t_mention_brand
1506,5fc0a7fc2a2dbfccb4728c29,https://pantip.com/topic/30199155,มุมกาแฟ NONแดง ยินดีต้อนรับเฉพาะผู้ที่จริงใจ ไ...,"[มุม, กาแฟ, non, แดง, ยินดีต้อนรับ, จริงใจ, ไม...",{'รสจืด'},set()
1508,5fc0a7fc2a2dbfccb4728c2b,https://pantip.com/topic/30059506,อยากทราบว่าคุณแม่ที่คลอดลูกแล้ว\nตอนท้อง บำรุง...,"[ทราบ, คุณแม่, คลอดลูก, ตอน, ท้อง, บำรุง, อาหา...",{'นม'},set()
1509,5fc0a7fc2a2dbfccb4728c2c,https://pantip.com/topic/30007179,กำลังจะเป็นคุณแม่มือใหม่ค่ะ ทำตัวไม่ถูกเลยค่ะ\...,"[คุณแม่มือใหม่, ทำตัว, ไม่, ทราบ, ที่จะ, กินนม...",{'กินนม'},set()
1510,5fc0a7fc2a2dbfccb4728c2d,https://pantip.com/topic/30020409,พรุ่งนี้ลูกชายอายุครบ 1 ขวบ 3 เดือน หม่ำนมแม่จ...,"[พรุ่งนี้, ลูกชาย, อายุ, ขวบ, เดือน, หม่ำ, นม,...",{'นม'},set()
1513,5fc0a7fc2a2dbfccb4728c30,https://pantip.com/topic/30651064,อยากรู้ว่าปัจจุบันพวกนมกล่องทึ่ขายใน 7-11 หรือ...,"[อยากรู้, นมกล่อง, ทึ่, ขาย, 7-Eleven, ห้างสรร...","{'นมวัว', 'นม', 'นมกล่อง', 'น้ำนมโค'}",set()


In [87]:
df_join_url = pd.merge(df_urls_milk,df_cmt_process,how='inner',on='URLs')
df_join_url.drop(columns=['Table','_id_x','_id_y','text_x','token_text_x','cmt_mention_brand','cmt_mention_product'],inplace=True)
df_join_url.columns = ['URLs','t_mention_brand','t_mention_product'
                       ,'Retrived_date','commentId','comment_text','token_text',]

In [88]:
df_join_url.head(3)

,URLs,t_mention_brand,t_mention_product,Retrived_date,commentId,comment_text,token_text
0,https://pantip.com/topic/39868603,"{'รสหวาน', 'นม'}",{'เมจิ'},1,เมื่อก่อนหลายปีมาแล้ว ในเซเว่นยังมีนมเมจิรสหวา...,2020-10-26 12:40:26.135083,"[ปี, 7-Eleven, ยังมี, นม, เมจิ, รสหวาน, ขาย, อ..."
1,https://pantip.com/topic/39868603,"{'รสหวาน', 'นม'}",{'เมจิ'},2,เบาหวานครับ แฮ่ๆ,2020-10-26 12:40:26.135083,[เบาหวาน]
2,https://pantip.com/topic/39868603,"{'รสหวาน', 'นม'}",{'เมจิ'},3,หาซื้อไม่ได้นี่คิดได้ 2 อย่างนะ. คนไม่ค่อย...,2020-10-26 12:40:26.135083,"[หา, ซื้อไม่ได้, คิดได้, , . คน, กินกัน, ไม..."


EDA#2

In [89]:
df_view = df_urls_milk[['URLs','t_mention_product']]
df_view.column = ['URLs','t_mention_product']
df_view_count = df_view.groupby('t_mention_product').count()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [94]:
df_c_urls_milk = df_cmt_process[(df_cmt_process.cmt_mention_product.str.len() != 0) | (df_thr_process.URLs.isin(execption_urls))]
df_c_urls_milk['cmt_mention_brand'] = df_c_urls_milk['cmt_mention_brand'].apply(lambda x: repr(set(x)))
df_c_urls_milk['cmt_mention_product'] = df_c_urls_milk['cmt_mention_product'].apply(lambda x: repr(set(x)))

In [95]:
df_view2 = df_c_urls_milk[['URLs','cmt_mention_product']]
df_view2.column = ['URLs','cmt_mention_product']
df_view_count2 = df_view2.groupby('cmt_mention_product').count()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [99]:
df_urls_milk.tail()

,_id,URLs,text,token_text,t_mention_product,t_mention_brand
1506,5fc0a7fc2a2dbfccb4728c29,https://pantip.com/topic/30199155,มุมกาแฟ NONแดง ยินดีต้อนรับเฉพาะผู้ที่จริงใจ ไ...,"[มุม, กาแฟ, non, แดง, ยินดีต้อนรับ, จริงใจ, ไม...",{'รสจืด'},set()
1508,5fc0a7fc2a2dbfccb4728c2b,https://pantip.com/topic/30059506,อยากทราบว่าคุณแม่ที่คลอดลูกแล้ว\nตอนท้อง บำรุง...,"[ทราบ, คุณแม่, คลอดลูก, ตอน, ท้อง, บำรุง, อาหา...",{'นม'},set()
1509,5fc0a7fc2a2dbfccb4728c2c,https://pantip.com/topic/30007179,กำลังจะเป็นคุณแม่มือใหม่ค่ะ ทำตัวไม่ถูกเลยค่ะ\...,"[คุณแม่มือใหม่, ทำตัว, ไม่, ทราบ, ที่จะ, กินนม...",{'กินนม'},set()
1510,5fc0a7fc2a2dbfccb4728c2d,https://pantip.com/topic/30020409,พรุ่งนี้ลูกชายอายุครบ 1 ขวบ 3 เดือน หม่ำนมแม่จ...,"[พรุ่งนี้, ลูกชาย, อายุ, ขวบ, เดือน, หม่ำ, นม,...",{'นม'},set()
1513,5fc0a7fc2a2dbfccb4728c30,https://pantip.com/topic/30651064,อยากรู้ว่าปัจจุบันพวกนมกล่องทึ่ขายใน 7-11 หรือ...,"[อยากรู้, นมกล่อง, ทึ่, ขาย, 7-Eleven, ห้างสรร...","{'นมวัว', 'นม', 'นมกล่อง', 'น้ำนมโค'}",set()


In [101]:
print('จำนวนกระทู้',df_thr_process.shape[0])
print('จำนวน comment',df_cmt_process.shape[0])
print('จำนวนกระทู้พูดถึงนม',df_urls_milk.shape[0])
print('จำนวน comment พูดถึงนม',df_c_urls_milk.shape[0])
print('หมายเหตุ : ไม่ได้ impute ขั้วกระทู้เข้า comment')

จำนวนกระทู้ 1514
จำนวน comment 60545
จำนวนกระทู้พูดถึงนม 767
จำนวน comment พูดถึงนม 8962
หมายเหตุ : ไม่ได้ impute ขั้วกระทู้เข้า comment


In [103]:
#df_view_count2.sort_values(by='URLs',ascending=False)

### 4.ทำ Topic Modeling และคัดเลือกกระทู้ที่มีความเกี่ยวข้อง แล้วดึงคำไปใช้

In [ ]:
dictionary = gensim.corpora.Dictionary(df_join_url['token_text'])
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in df_join_url['token_text']]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

In [ ]:
#num_topics = 9
chunksize = 4000                  # size of the doc looked at every pass
iterations = 50
eval_every = 1                    # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize \
                                     ,alpha='auto', eta='auto',iterations=iterations, eval_every=eval_every)
                                     #,num_topics=num_topics \
                                     #, eval_every=eval_every)

In [ ]:
pyLDAvis.gensim.prepare(model, gensim_corpus, dictionary, R=40, lambda_step=0.1)

### 5.get_term_topics เทียบคำที่ได้ไป Review Literature ตามค่าความน่าจะเป็น แล้วสรุป Term

https://radimrehurek.com/gensim/models/ldamodel.html

<b>Domain ที่พบหลักๆ นอกเหนือจากตัว Product ห้างที่ขาย แบรนด์ (อย่างไรก็ตามต้องทำ LDA Vis อีกรอบเพื่อจัดกลุ่มใหม่หลังคัดเลือก Topic/Term)</b><br>
ค้นหาเทียบเคียงกับ ปัจจัยที่มีผลต่อการบริโภคนม
1. แม่และเด็ก (Kurajdova et al, 2015)
2. ออกกำลังกาย เล่นเวท วิ่ง พูดถึง Whey หางนม โปรตีน เป็นหลัก (คิดว่าแตกจากหมวดสุขภาพ)
3. โปรโมชั่น เช่น จัดโปร ส่วนลด พันแถม แพ๊กคู่ (สุภชาติ ชัยณรงค์สิงห์, 2539)
4. เอานมให้หมาแมวกิน (เทรนด์ใหม่)
5. รสชาติและคุณภาพ เช่น หอม เหม็น มัน อร่อย บูด เสีย GMP (ชาลิสา สถีระกานนท์, 2560)
6. ความยากง่ายในการหาซื้อ (Aen De Alwis et al, 2009 & สุภชาติ ชัยณรงค์สิงห์, 2539)
7. ทำกาแฟ ทำขนม (Kurajdova et al, 2015)
8. สุขภาพ ความสูง ส่วนผสมเพื่อสุขภาพ การขับถ่าย อนุมูลอิสระ (Kurajdova et al, 2015 & Aen De Alwis et al, 2009 & สุภชาติ ชัยณรงค์สิงห์, 2539)
9. ราคาและความคุ้มค่า (ปิยฉัตร ช่างเหล็ก, 2561)
<br><br>ตรงนี้ต้องหา Paper มาช่วยยันในบางหัวข้อ ถ้ามีส่วนเกินถือว่าเป็นบริบทของประเทศไทย

### 6.สรุป Term & reduced_keyword

In [104]:
attrributes = ['การบูด','โปร','โปรโมชั่น','แถม','บูด','เป็นผลดี','หา','หมดอายุ', 'กลิ่น','หอม','กลิ่นหอม', 'แนะนำ', 'กิน', 'shelf life','แม่','คุณแม่'
 ,'ลูก','เด็ก','เลี้ยงลูก','เลี้ยงเด็ก','พัฒนาการ','คุณแม่มือใหม่','สั่ง', 'ค่าใช้จ่าย', 'เพิ่มขึ้น', 'สินค้า', 'ชอบ','นิยม', 'สี', 'ซื้อไม่ได้'
 , 'ทิ้ง', 'จืด','หวาน' , 'fat', 'นิยม','ราคาสูง','กาแฟ','ชงกาแฟ','หาย','ต้ม','อุ่น','ร้อน', 'พื้นฐาน', 'ขายไม่ดี','สูง','ส่วนสูง'
 , 'กาแฟสด', 'จ่าย', 'ไขมันต่ำ', 'นม', 'ขาย', 'รส','รสชาติ', 'รสหวาน','กลมกล่อม','ไขมัน','พนักงาน','ไมโครเวฟ','ฟาร์ม'
 ,'ส่วนผสม','ผสม','เบาหวาน','รสพื้นฐาน','โปรตีน','เวย์','เวท','วิ่ง','protein','whey','ขับถ่าย','อึ','dha','น้ำตาล','เบาหวาน'
 ,'ฝา', 'นมสด','ถ้วย', 'ร้าน', 'ดื่ม', 'เซเว่น', 'ขวด', 'นึกถึง', 'เสียใจ', 'แตก', 'ซื้อ','ขาย','มีลูก','เทรนเนอร์'
 ,'ขายไม่ดี','แพคคู่','ค่าจัดส่ง','shelf life','พนักงานขายนม','แพ้นม','แพ้นมวัว','เล่นเวท','ฝาน้ำเงิน','ฝาสีเขียว'
 ,'นมอุ่น','ชานม','กินนม','ดื่มนม','ท้องเสีย','ลูกสุนัข','สุนัข','หมา','แมว','คายทิ้ง','เจมส์จิ','แป้ง','ลดราคา'
 ,'ซื้อประจำ','ซื้อไม่ได้','คาปูชิโน่','อเมริกาโน่','ร้านนม','whey formula','ผิดสังเกต','เสียความรู้สึก','ชี้แจง','บำรุง','น้ำผึ้ง'
 ,'อาหารเสริม','มีประโยชน์','วิตามิน','นมผง','แคลเซี่ยม','ท้อง','แพะ','อร่อย','โภชนาการ','โรงเรียน','พ่อแม่','ครู','ยูเอชที'
 ,'ผู้บริโภค','ขโมย','ไอโอดีน','นมข้นหวาน','เนย','ตรวจสอบ','แกลลอน','มันดี','นมวัว','อาเจียน','เวฟ','ไมโครเวฟ','สารอาหาร'
 ,'ขนส่ง','ถูก','แพง','เค้ก','ทิ้ง','วันหมดอายุ','โอเมก้า','กล่อง','พลังงาน','โภชนาการ','ขนมปัง','ของแถม','ราคาสูง','น้ำนมโค'
 ,'บรรจุภัณฑ์','นมถั่วเหลือง','ความแข็งแรง','แข็งแรง','พรีเซ็นเตอร์','ญี่ปุ่น','น้ำผลไม้','ออกกำลังกาย','ประหยัด','วิปปิ้ง','uht','เชื้อจุลินทรีย์'
 ,'แลคโตส']

products = ['สตรอว์เบอร์รี','ช็อกโกแลต','กาแฟ','รสหวาน','รสจืด','รสกาแฟ','ไขมันต่ำ','ไขมัน 0%','ไฮโปรตีน','อัลมอนด์'
,'รสกล้วย','grass fed','นมฟรีแลคโตส','พาสเจอร์ไรส์','เมจิโกลด์','นมฮอกไกโด','เบดไทม์','ดาร์คช็อกโกแลต','ไฮแคลเซียม'
,'คาราเมล','มอลต์','เมล่อน','ชาเขียวมัจฉะ','บัลแกเรีย','โยเกิร์ต','รสธรรมชาติ','รสกลมกล่อม','โยเกิร์ต','ซากุระ','โคล่า'
,'แตงโม']

brands = ['ดัชมิลล์','เมจิ','โฟร์โมสต์','โชคชัย','แดรี่โฮม','เอ็มมิลค์','แมคโนเลีย','ไทยเดนมาร์ค'
          ,'หนองโพ','คาเนชั่น','บีทาเก้น','จิตรลดา']
stores = ['tops','makro','lotus','bigc','7-Eleven']

reduceCol = attrributes + products + stores

In [105]:
len(reduceCol)

211

In [106]:
def reduced_keyword(wtkn,redCol):
    del_list = []
    ret_wtkn = wtkn
    for each in ret_wtkn:
        if each not in redCol:
            del_list.append(each)
    ret_wtkn = [x for x in ret_wtkn if x not in del_list]
    return ret_wtkn

### 7.สร้าง (Custom) Bag of word ด้วย dictionary.doc2bow จัดลง dataframe

In [107]:
cursor_thread = col_thread.find()
df_thr_process = pd.DataFrame(cursor_thread)

In [108]:
cursor_comment = col_comment.find()
df_cmt_process = pd.DataFrame(cursor_comment)

In [109]:
df_urls_milk = df_thr_process[(df_thr_process.t_mention_product.str.len() != 0) | (df_thr_process.URLs.isin(execption_urls))]
df_urls_milk['t_mention_brand'] = df_urls_milk['t_mention_brand'].apply(lambda x: repr(x))
df_urls_milk['t_mention_product'] = df_urls_milk['t_mention_product'].apply(lambda x: repr(x))

In [110]:
df_join_url = pd.merge(df_urls_milk,df_cmt_process,how='inner',on='URLs')
df_join_url.drop(columns=['Table','_id_x','_id_y','text_x','token_text_x','cmt_mention_brand','cmt_mention_product'],inplace=True)
df_join_url.columns = ['URLs','t_mention_product','t_mention_brand','commentId'
                       ,'comment_text','Retrived_date','token_text',]

In [111]:
df_join_url['token_text_reduce'] = df_join_url['token_text'].apply(lambda x: reduced_keyword(x,reduceCol))

In [112]:
df_join_url.set_index('commentId',inplace=True)
df_join_url.head(2)

,URLs,t_mention_product,t_mention_brand,comment_text,Retrived_date,token_text,token_text_reduce
commentId,,,,,,,
1,https://pantip.com/topic/39868603,"['นม', 'รสหวาน', 'นม', 'รสหวาน']",['เมจิ'],เมื่อก่อนหลายปีมาแล้ว ในเซเว่นยังมีนมเมจิรสหวา...,2020-10-26 12:40:26.135083,"[ปี, 7-Eleven, ยังมี, นม, เมจิ, รสหวาน, ขาย, อ...","[7-Eleven, นม, รสหวาน, ขาย, หาย, หา, tops, ขวด..."
2,https://pantip.com/topic/39868603,"['นม', 'รสหวาน', 'นม', 'รสหวาน']",['เมจิ'],เบาหวานครับ แฮ่ๆ,2020-10-26 12:40:26.135083,[เบาหวาน],[เบาหวาน]


In [113]:
dictionary = gensim.corpora.Dictionary(df_join_url['token_text_reduce'])

In [114]:
df_join_url['reduce_bow'] = df_join_url["token_text_reduce"].map(dictionary.doc2bow)
df_join_url['reduce_bow_txt'] = df_join_url["reduce_bow"].apply(lambda x:[(dictionary[id_], frequence) for id_, frequence in x])

In [115]:
def freq_brand(x):
    e = eval(x)
    e.sort()
    f = [(k,len(list(g))) for k, g in groupby(e)]
    return f

In [116]:
df_join_url['t_mention_product'] = df_join_url['t_mention_product'].apply(lambda x: freq_brand(x))
df_join_url['t_mention_brand'] = df_join_url['t_mention_brand'].apply(lambda x: freq_brand(x))

In [117]:
df_join_url.head()

,URLs,t_mention_product,t_mention_brand,comment_text,Retrived_date,token_text,token_text_reduce,reduce_bow,reduce_bow_txt
commentId,,,,,,,,,
1,https://pantip.com/topic/39868603,"[(นม, 2), (รสหวาน, 2)]","[(เมจิ, 1)]",เมื่อก่อนหลายปีมาแล้ว ในเซเว่นยังมีนมเมจิรสหวา...,2020-10-26 12:40:26.135083,"[ปี, 7-Eleven, ยังมี, นม, เมจิ, รสหวาน, ขาย, อ...","[7-Eleven, นม, รสหวาน, ขาย, หาย, หา, tops, ขวด...","[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 2...","[(7-Eleven, 1), (tops, 1), (ขวด, 2), (ขาย, 1),..."
2,https://pantip.com/topic/39868603,"[(นม, 2), (รสหวาน, 2)]","[(เมจิ, 1)]",เบาหวานครับ แฮ่ๆ,2020-10-26 12:40:26.135083,[เบาหวาน],[เบาหวาน],"[(12, 1)]","[(เบาหวาน, 1)]"
3,https://pantip.com/topic/39868603,"[(นม, 2), (รสหวาน, 2)]","[(เมจิ, 1)]",หาซื้อไม่ได้นี่คิดได้ 2 อย่างนะ. คนไม่ค่อย...,2020-10-26 12:40:26.135083,"[หา, ซื้อไม่ได้, คิดได้, , . คน, กินกัน, ไม...","[หา, ซื้อไม่ได้, นิยม, ขาย, ซื้อ, กิน, รสหวาน]","[(3, 1), (6, 1), (9, 1), (10, 1), (13, 1), (14...","[(ขาย, 1), (นิยม, 1), (รสหวาน, 1), (หา, 1), (ก..."
4,https://pantip.com/topic/39868603,"[(นม, 2), (รสหวาน, 2)]","[(เมจิ, 1)]",เป็นรสที่ขายไม่ดีครับ\r\nรสพื้นฐานคือ\r\nจืด แ...,2020-10-26 12:40:26.135083,"[รส, ขายไม่ดี, รส, พื้นฐาน, จืด, แบ่ง, %,, ไขม...","[รส, ขายไม่ดี, รส, พื้นฐาน, จืด, ไขมันต่ำ, ไขม...","[(4, 1), (7, 1), (8, 2), (9, 1), (16, 1), (17,...","[(ขายไม่ดี, 1), (พื้นฐาน, 1), (รส, 2), (รสหวาน..."
5,https://pantip.com/topic/39868603,"[(นม, 2), (รสหวาน, 2)]","[(เมจิ, 1)]",ถ้าขายไม่ดีเค้าก็เอาออกครับ shelf life มันน้อย...,2020-10-26 12:40:26.135083,"[ขายไม่ดี, เค้า, shelf life, เวลา, หมดอายุ, ค่...","[ขายไม่ดี, shelf life, หมดอายุ, ค่าใช้จ่าย, ร้...","[(2, 1), (4, 2), (14, 1), (24, 1), (25, 1), (2...","[(ขวด, 1), (ขายไม่ดี, 2), (ซื้อ, 1), (shelf li..."


Create cooccurence matrix

In [119]:
df_brand_cooc = pd.concat([pd.DataFrame(s,columns=['w','c']).set_index('w') for s in df_join_url["t_mention_brand"]], axis=1, sort=False).fillna(0).T.set_index(df_join_url.index)
df_cooc = pd.concat([pd.DataFrame(s,columns=['w','c']).set_index('w') for s in df_join_url["reduce_bow_txt"]], axis=1, sort=False).fillna(0).T.set_index(df_join_url.index)

In [120]:
df_brand_cooc.reset_index(inplace=True)
df_cooc.reset_index(inplace=True)

In [121]:
join_df = pd.merge(df_brand_cooc, df_cooc, on='commentId', how='inner')
join_df.set_index('commentId',inplace=True)

In [122]:
join_df.head()

,เมจิ,ดัชมิลล์,แมคโนเลีย,ไทยเดนมาร์ค,หนองโพ,บีทาเก้น,คาเนชั่น,โฟร์โมสต์,เอ็มมิลค์,จิตรลดา,...,dha,พรีเซ็นเตอร์,ความแข็งแรง,เจมส์จิ,ของแถม,เสียความรู้สึก,ผิดสังเกต,ไอโอดีน,เบดไทม์,คุณแม่มือใหม่
commentId,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Create frequency co-occurrence matrix จาก Bag of word

In [123]:
item_item_matrix = pd.DataFrame(index=join_df.columns,columns=join_df.columns).fillna(0)

In [124]:
def cnt_Grpby(df,i,j):
    PAIB = pd.DataFrame(df.groupby(df.iloc[:,i].name)[df.iloc[:,j].name].value_counts())
    PAIB.index.names = ['A','B']
    PAIB.columns = ['freq']
    PAIB.reset_index(inplace=True)
    _PAIB = PAIB[(PAIB.A>0)&(PAIB.B>0)]
    return _PAIB.freq.sum()

In [125]:
# Co-occurrence
for i in range(0,len(item_item_matrix.columns)):
    for j in range(0,len(item_item_matrix.columns)):
        if i != j:
            item_item_matrix.iloc[i,j] = cnt_Grpby(join_df,i,j)
        else:
            item_item_matrix.iloc[i,j] = 0

In [126]:
item_item_matrix.head()

,เมจิ,ดัชมิลล์,แมคโนเลีย,ไทยเดนมาร์ค,หนองโพ,บีทาเก้น,คาเนชั่น,โฟร์โมสต์,เอ็มมิลค์,จิตรลดา,...,dha,พรีเซ็นเตอร์,ความแข็งแรง,เจมส์จิ,ของแถม,เสียความรู้สึก,ผิดสังเกต,ไอโอดีน,เบดไทม์,คุณแม่มือใหม่
เมจิ,0,761,50,179,196,14,50,375,2,0,...,0,11,1,32,1,5,1,0,0,0
ดัชมิลล์,761,0,0,195,311,9,20,487,2,152,...,2,3,2,3,33,2,0,1,1,0
แมคโนเลีย,50,0,0,0,0,0,0,13,0,0,...,0,0,0,0,0,0,0,0,0,0
ไทยเดนมาร์ค,179,195,0,0,270,0,0,282,0,237,...,3,0,1,0,0,0,0,0,0,2
หนองโพ,196,311,0,270,0,0,0,355,0,169,...,0,0,1,0,0,0,0,0,0,2


In [127]:
item_item_matrix.to_excel(comment_cooc_freq_file, index=True)

### Create co-occurrence matrix with Lift normalization

In [128]:
item_item_matrix = pd.DataFrame(index=join_df.columns,columns=join_df.columns).fillna(0)

In [129]:
def a_in_b(df,i,j):
    PAIB = pd.DataFrame(df.groupby(df.iloc[:,j].name)[df.iloc[:,i].name].value_counts())
    PAIB.index.names = ['B','A']
    PAIB.columns = ['freq']
    PAIB.reset_index(inplace=True)
    PAIB_ = PAIB[(PAIB.A>0)&(PAIB.B>0)]
    return PAIB_.freq.sum()/PAIB.freq.sum()

In [130]:
for i in range(0,len(item_item_matrix.columns)) :
    for j in range(0,len(item_item_matrix.columns)) :
        A = join_df.iloc[:,i]
        B = join_df.iloc[:,j]
        PA = A[A!=0].count()/A.shape[0]
        PB = B[B!=0].count()/B.shape[0]
        PAB = PB*a_in_b(join_df,i,j)
        item_item_matrix.iloc[i,j] = PAB/(PA*PB)

In [131]:
item_item_matrix.replace([np.inf, -np.inf], np.nan,inplace=True)
item_item_matrix.fillna(0,inplace=True)

In [132]:
item_item_matrix.head()

,เมจิ,ดัชมิลล์,แมคโนเลีย,ไทยเดนมาร์ค,หนองโพ,บีทาเก้น,คาเนชั่น,โฟร์โมสต์,เอ็มมิลค์,จิตรลดา,...,dha,พรีเซ็นเตอร์,ความแข็งแรง,เจมส์จิ,ของแถม,เสียความรู้สึก,ผิดสังเกต,ไอโอดีน,เบดไทม์,คุณแม่มือใหม่
เมจิ,1.000000,0.147338,0.009681,0.034656,0.037948,0.002711,0.009681,0.072604,0.000387,0.000000,...,0.000000,0.002130,0.000194,0.006196,0.000194,0.000968,0.000194,0.000000,0.000000,0.000000
ดัชมิลล์,0.309853,1.000000,0.000000,0.079397,0.126629,0.003664,0.008143,0.198290,0.000814,0.061889,...,0.000814,0.001221,0.000814,0.001221,0.013436,0.000814,0.000000,0.000407,0.000407,0.000000
แมคโนเลีย,0.793651,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.206349,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ไทยเดนมาร์ค,0.309154,0.336788,0.000000,1.000000,0.466321,0.000000,0.000000,0.487047,0.000000,0.409326,...,0.005181,0.000000,0.001727,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003454
หนองโพ,0.342657,0.543706,0.000000,0.472028,1.000000,0.000000,0.000000,0.620629,0.000000,0.295455,...,0.000000,0.000000,0.001748,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003497


In [133]:
item_item_matrix.to_excel(comment_cooc_lift_file, index=True)